In [1]:
import sys
import ROOT
import numpy as np
import pandas as pd
import root_pandas
import seaborn as sb
import matplotlib.pyplot as plt
import uproot
import time

from itertools import product

#from root_numpy import root2array

from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc

from bayes_opt import BayesianOptimization
from sklearn.preprocessing import QuantileTransformer
import pickle

Welcome to JupyROOT 6.16/00


/Users/jonamotta/anaconda3/envs/python_root/lib/python2.7/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.3 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)
Using TensorFlow backend.


In [2]:
#########################################
### PREPARE DFs FOR TRAINING AND TEST ###
#########################################


features = [
    'unpairedMuE_muSys_rf', # unpaired muon energy in the rest frame of the jpsi muons sys
    'reco_pT_diff', # difference between the jpsi muons pT
    'jpsi_mass_diffReco', # mu_sys.Mag() - jpsi_mass
    'mu_pt'     ,
    'mu_eta'    ,
    'mu_phi'    ,
    'mu1_pt'    ,
    'mu1_eta'   ,
    'mu1_phi'   ,
    'mu2_pt'    ,
    'mu2_eta'   ,
    'mu2_phi'   ,
]

mu_events = pd.read_csv('mu_events_bkgDiscr.csv')
tau_events = pd.read_csv('tau_events_bkgDiscr.csv')
bkg_events = pd.read_csv('bkg_events_bkgDiscr.csv')

# concatenate the two samples
dataset = pd.concat([mu_events, tau_events, bkg_events], sort=False)

# shuffle and split train/test
train, test = train_test_split(dataset, test_size=0.85, random_state=1986, shuffle=True)

# X and Y on the training sample
X = pd.DataFrame(train, columns=features)
Y = pd.DataFrame(train, columns=['target'])

In [3]:
##################
### PREPROCESS ###
##################


qt = QuantileTransformer(output_distribution='normal', random_state=1986)
qt.fit(X[features])
transformedX = qt.transform(X[features])
pickle.dump( qt, open( 'quantile_tranformation.pck', 'w' ) )

In [4]:
####################################################
### FUNCTION FOR BAYESIAN OPTIMIZATION OF THE NN ###
####################################################


def BO_function(features,transformedX,Y,test,pbounds,init_points,n_iter):
    start = time.time()
    def NN_function(n_layers,units_perlayer,batch_size):
    
        #optimizer_fcts = ['nadam', 'adamax', 'adam', 'adadelta', 'adagrad', 'rmsprop', 'sgd']
        #activation_fcts = ['softmax', 'elu', 'selu', 'relu', 'softplus', 'softsign', 'tanh', 'sigmoid', 
        #                   'hard_sigmoid', 'exponential']
        #metrics_fcts = ['binary_accuracy', 'categorical_accuracy', 'sparse_categorical_accuracy', 
        #                'top_k_categorical_accuracy', 'sparse_top_k_categorical_accuracy']
        #loss_fcts = ['mean_squared_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
        #            'squared_hinge', 'hinge', 'categorical_hinge', 'logcosh', 'categorical_crossentropy',
        #            'sparse_categorical_crossentropy', 'binary_crossentropy', 'kullback_leibler_divergence',
        #            'poisson', 'cosine_proximity']
        
        # I want units_perlayer to be a multiple of 2 -> I always take le lower multiple of 2 starting from the
        # float that the algorith is giving me
        if units_perlayer%2 < 1:
            units_perlayer = int(units_perlayer)
        else:
            units_perlayer = int(units_perlayer-1)

        # define the model
        model = Sequential()
        for i in range(int(n_layers)):
            model.add(Dense(units_perlayer, input_dim=len(features),activation='relu'))
        model.add(Dense(1,activation='sigmoid'))

        # compile the model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        # fit the model
        model.fit(transformedX, Y, epochs=20, batch_size=int(batch_size), validation_split=0.6,verbose=0)

        # evaluate the model
        scores = model.evaluate(transformedX, Y,verbose=0)

        # calculate predictions on the test sample
        x = pd.DataFrame(test, columns=features)
        qt = pickle.load(open( 'quantile_tranformation.pck', 'r' ))
        transformedx = qt.transform(x[features])
        y = model.predict(transformedx)
    
        # create this random in order to avoid repetitions in the insertion of the score of the NN (nnS)
        k = np.random.normal(100,20)
        
        # add the score to the test sample dataframe
        test.insert(len(test.columns), 'nnS'+str(k), y)

        # let sklearn do the heavy lifting and compute the ROC curves for you
        fpr, tpr, wps = roc_curve(test.target, test['nnS'+str(k)])

        # compute the auc
        auroc = auc(fpr, tpr)

        # compute Gini index
        gini_index = (auroc-0.5)*2
        
        return np.log((auroc*gini_index*scores[1])**4)

    optimizer = BayesianOptimization(
        f = NN_function,
        pbounds = pbounds,
    )
    
    # optimize
    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
        alpha = 1e-3
    )
    
    print optimizer.max
    
    end = time.time()
    print 'Running time of the Bayesian Optimization = %.1f'%(end - start)

In [5]:
#########################################
### OPTIMIZE THE PARAMETERS OF THE NN ###
#########################################


# bounded region of parameter space
pbounds = {'n_layers': (1, 10), 'units_perlayer': (16, 128), 'batch_size': (5, 5000)}
# , 'optimizer': (0,7), 'activation': (0,10), 'loss': (0,13)




# call the function that creates the optimizer
BO_function(features,transformedX,Y,test,pbounds,40,60)

|   iter    |  target   | batch_... | n_layers  | units_... |
-------------------------------------------------------------
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
|  1        | -0.9205   |  1.939e+0 |  8.997    |  123.7    |
|  2        | -1.166    |  2.122e+0 |  3.692    |  107.8    |
|  3        | -1.249    |  4.629e+0 |  8.844    |  88.74    |
|  4        | -0.9167   |  109.2    |  8.474    |  64.48    |
|  5        | -1.216    |  3.495e+0 |  9.124    |  76.42    |
|  6        | -2.783    |  1.509e+0 |  2.053    |  45.68    |
|  7        | -2.42     |  3.239e+0 |  2.849    |  90.25    |
|  8        | -3.056    |  3.659e+0 |  2.568    |  94.47    |
|  9        | -2.278    |  2.789e+0 |  4.226    |  31.96    |
|  10       | -0.9467   |  612.2    |  8.067    |  66.94    |
|  11       | -1.118    |  3.859e+0 |  9.161    |  110.1    |
|  12       | -8.212    |  2.984e+0 |  1.056    |  18.12    |
|  13       

|  74       | -1.261    |  109.3    |  1.0      |  128.0    |
|  75       | -0.9343   |  1.7e+03  |  10.0     |  128.0    |
|  76       | -2.346    |  1.34e+03 |  1.0      |  127.6    |
|  77       | -1.831    |  305.8    |  1.0      |  72.63    |
|  78       | -0.938    |  994.8    |  10.0     |  128.0    |
|  79       | -1.23     |  2.678e+0 |  10.0     |  49.1     |
|  80       | -0.9288   |  2.878e+0 |  10.0     |  128.0    |
|  81       | -1.208    |  404.6    |  10.0     |  16.0     |
|  82       | -3.092    |  2.481e+0 |  1.0      |  128.0    |
|  83       | -4.646    |  1.631e+0 |  1.0      |  16.0     |
|  84       | -2.86     |  1.675e+0 |  1.0      |  95.21    |
|  85       | -2.021    |  566.6    |  1.0      |  88.94    |
|  86       | -0.9511   |  1.902e+0 |  8.368    |  109.8    |
|  87       | -0.9829   |  148.6    |  8.057    |  81.24    |
|  88       | -0.9316   |  732.1    |  9.388    |  89.18    |
|  89       | -0.8794   |  716.3    |  5.685    |  128.0    |
|  90   

2019-07-18 10:31:50.781974: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
